## Section to set gpu usage and create a session

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

## all the imports

In [2]:
!pip install matplotlib


     |████████████████████████████████| 13.1MB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 92kB 2.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 2.0MB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl


In [2]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.optimizers import Adam,SGD
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Lambda
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers.merge import add, concatenate
from keras_preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.utils import to_categorical,Sequence
import pandas as pd
import math
import sincnet
from tensorflow import set_random_seed
from keras import models, layers
import numpy as np
import sincnet
from keras.layers import Dense, Dropout, Activation
from keras.layers import MaxPooling1D,MaxPooling2D, Conv1D, LeakyReLU, BatchNormalization, Dense, Flatten
from keras.layers import InputLayer, Input
from keras.models import Model
# from tensorflow.python.keras.utils.data_utils import Sequence
# import pydot

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

## Function to do framing and windowing over the audio to get a 2D output that can feeded to the WCNN

In [7]:
def test(signal):
    pre_emphasis = 0.97
    frame_size = 256
    frame_stride = 128
    nfilt = 20
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    frame_length, frame_step = frame_size, frame_stride  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
#     print(num_frames)
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    frames *= np.hamming(frame_length)
    print(frames.shape)
    return frames

## Section to perform wavelet decomposition

In [5]:
# batch operation usng tensor slice
def WaveletTransformAxisY(batch_img):
    odd_img  = batch_img[:,0::2]
    even_img = batch_img[:,1::2]
    L = (odd_img + even_img) / 2.0
    H = K.abs(odd_img - even_img)
    return L, H

def WaveletTransformAxisX(batch_img):
    # transpose + fliplr
    tmp_batch = K.permute_dimensions(batch_img, [0, 2, 1])[:,:,::-1]
    _dst_L, _dst_H = WaveletTransformAxisY(tmp_batch)
    # transpose + flipud
    dst_L = K.permute_dimensions(_dst_L, [0, 2, 1])[:,::-1,...]
    dst_H = K.permute_dimensions(_dst_H, [0, 2, 1])[:,::-1,...]
    return dst_L, dst_H

In [6]:
def Wavelet(batch_image):
    # make channel first image
    batch_image = K.permute_dimensions(batch_image, [0, 3, 1, 2])
    r = batch_image[:,0]
    print(r.shape)

    # level 1 decomposition
    wavelet_L, wavelet_H = WaveletTransformAxisY(r)
    r_wavelet_LL, r_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    r_wavelet_HL, r_wavelet_HH = WaveletTransformAxisX(wavelet_H)


    wavelet_data = [r_wavelet_LL, r_wavelet_LH, r_wavelet_HL, r_wavelet_HH] 
#                    
    transform_batch = K.stack(wavelet_data, axis=1)

    # level 2 decomposition
    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(r_wavelet_LL)
    r_wavelet_LL2, r_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    r_wavelet_HL2, r_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)


    wavelet_data_l2 = [r_wavelet_LL2, r_wavelet_LH2, r_wavelet_HL2, r_wavelet_HH2]
#                     
    transform_batch_l2 = K.stack(wavelet_data_l2, axis=1)

    # level 3 decomposition
    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(r_wavelet_LL2)
    r_wavelet_LL3, r_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    r_wavelet_HL3, r_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)


    wavelet_data_l3 = [r_wavelet_LL3, r_wavelet_LH3, r_wavelet_HL3, r_wavelet_HH3]
#                     
    transform_batch_l3 = K.stack(wavelet_data_l3, axis=1)

    # level 4 decomposition
    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(r_wavelet_LL3)
    r_wavelet_LL4, r_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    r_wavelet_HL4, r_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)



    wavelet_data_l4 = [r_wavelet_LL4, r_wavelet_LH4, r_wavelet_HL4, r_wavelet_HH4]
#                     
    transform_batch_l4 = K.stack(wavelet_data_l4, axis=1)



    decom_level_1 = K.permute_dimensions(transform_batch, [0, 2, 3, 1])
    decom_level_2 = K.permute_dimensions(transform_batch_l2, [0, 2, 3, 1])
    decom_level_3 = K.permute_dimensions(transform_batch_l3, [0, 2, 3, 1])
    decom_level_4 = K.permute_dimensions(transform_batch_l4, [0, 2, 3, 1])
    

    return [decom_level_1, decom_level_2, decom_level_3, decom_level_4]


def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 12576, 40, 4]), tuple([None, 6288, 20, 4]), 
            tuple([None, 3144, 10, 4]), tuple([None, 1572, 5, 4])]

In [18]:
img_batch = K.zeros(shape=(8,25152,80, 1), dtype='float32')
Wavelet(img_batch)

(8, 25152, 80)


[<tf.Tensor 'transpose_25:0' shape=(8, 12576, 40, 4) dtype=float32>,
 <tf.Tensor 'transpose_26:0' shape=(8, 6288, 20, 4) dtype=float32>,
 <tf.Tensor 'transpose_27:0' shape=(8, 3144, 10, 4) dtype=float32>,
 <tf.Tensor 'transpose_28:0' shape=(8, 1572, 5, 4) dtype=float32>]

In [13]:
im_batch = K.zeros(shape=(8, 224, 224, 3), dtype='float32')
im_batch = K.permute_dimensions(im_batch, [0, 3, 1, 2])
r = im_batch[:,0]
print(r.shape)

(8, 224, 224)


## Wavelet CNN model architecture

In [21]:
def step_decay(epoch):
	initial_lrate = 0.1
	drop = 0.5
	epochs_drop = 10.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate

def get_wavelet_cnn_model():

    input_shape = 75673, 1

    input_ = Input(input_shape, name='the_input')

    wavelet = Lambda(Wavelet, Wavelet_out_shape, name='wavelet')
    input_l1, input_l2, input_l3, input_l4 = wavelet(input_)
    print(input_l1.shape)
    # print(input_l2)
    # print(input_l3)
    # print(input_l4)
    # level one decomposition starts
    conv_1 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1')(input_l1)
#     pool_1 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1)
    norm_1 = BatchNormalization(name='norm_1')(conv_1)
    relu_1 = Activation('relu', name='relu_1')(norm_1)

    conv_1_2 = Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_1_2')(relu_1)
    # pool_1_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1_2)
    norm_1_2 = BatchNormalization(name='norm_1_2')(conv_1_2)
    relu_1_2 = Activation('relu', name='relu_1_2')(norm_1_2)
    

    # level two decomposition starts
    conv_a = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_a')(input_l2)
    # pool_a = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_a)
    norm_a = BatchNormalization(name='norm_a')(conv_a)
    relu_a = Activation('relu', name='relu_a')(norm_a)
    print(relu_1_2.shape,relu_a.shape)
    
    # concate level one and level two decomposition
    concate_level_2 = concatenate([relu_1_2, relu_a])
    conv_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2')(concate_level_2)
#     pool_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2)
    norm_2 = BatchNormalization(name='norm_2')(conv_2)
    relu_2 = Activation('relu', name='relu_2')(norm_2)

    conv_2_2 = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_2_2')(relu_2)
    # pool_2_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2_2)
    norm_2_2 = BatchNormalization(name='norm_2_2')(conv_2_2)
    relu_2_2 = Activation('relu', name='relu_2_2')(norm_2_2)

    # level three decomposition starts 
    conv_b = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_b')(input_l3)
    # pool_b = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_b)
    norm_b = BatchNormalization(name='norm_b')(conv_b)
    relu_b = Activation('relu', name='relu_b')(norm_b)

    conv_b_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_b_2')(relu_b)
    # pool_b_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_b_2)
    norm_b_2 = BatchNormalization(name='norm_b_2')(conv_b_2)
    relu_b_2 = Activation('relu', name='relu_b_2')(norm_b_2)
    print(relu_2_2.shape,relu_b_2.shape)
    # concate level two and level three decomposition 
    concate_level_3 = concatenate([relu_2_2, relu_b_2])
    conv_3 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_3')(concate_level_3)
#     pool_3 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_3)
    norm_3 = BatchNormalization(name='nomr_3')(conv_3)
    relu_3 = Activation('relu', name='relu_3')(norm_3)

    conv_3_2 = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_3_2')(relu_3)
    # pool_3_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_3_2)
    norm_3_2 = BatchNormalization(name='norm_3_2')(conv_3_2)
    relu_3_2 = Activation('relu', name='relu_3_2')(norm_3_2)

    # level four decomposition start
    conv_c = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_c')(input_l4)
    # pool_c = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_c)
    norm_c = BatchNormalization(name='norm_c')(conv_c)
    relu_c = Activation('relu', name='relu_c')(norm_c)

    conv_c_2 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_c_2')(relu_c)
    # pool_c_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_c_2)
    norm_c_2 = BatchNormalization(name='norm_c_2')(conv_c_2)
    relu_c_2 = Activation('relu', name='relu_c_2')(norm_c_2)

    conv_c_3 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_c_3')(relu_c_2)
    # pool_c_3 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_c_3)
    norm_c_3 = BatchNormalization(name='norm_c_3')(conv_c_3)
    relu_c_3 = Activation('relu', name='relu_c_3')(norm_c_3)
    print(relu_3_2.shape,relu_c_3.shape)
    # concate level level three and level four decomposition
    concate_level_4 = concatenate([relu_3_2, relu_c_3])
    conv_4 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_4')(concate_level_4)
    # pool_4 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_4)
    norm_4 = BatchNormalization(name='norm_4')(conv_4)
    relu_4 = Activation('relu', name='relu_4')(norm_4)

    conv_4_2 = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_4_2')(relu_4)
    # pool_4_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_4_2)
    norm_4_2 = BatchNormalization(name='norm_4_2')(conv_4_2)
    relu_4_2 = Activation('relu', name='relu_4_2')(norm_4_2)

    conv_5_1 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_5_1')(relu_4_2)
    # pool_5_1 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_5_1)
    norm_5_1 = BatchNormalization(name='norm_5_1')(conv_5_1)
    relu_5_1 = Activation('relu', name='relu_5_1')(norm_5_1)

    pool_5_1 = AveragePooling2D(pool_size=(3, 3), strides=1, padding='same', name='avg_pool_5_1')(relu_5_1)
    flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

    fc_5 = Dense(2048, name='fc_5')(flat_5_1)
    norm_5 = BatchNormalization(name='norm_5')(fc_5)
    relu_5 = Activation('relu', name='relu_5')(norm_5)
    drop_5 = Dropout(0.5, name='drop_5')(relu_5)

    fc_6 = Dense(2048, name='fc_6')(drop_5)
    norm_6 = BatchNormalization(name='norm_6')(fc_6)
    relu_6 = Activation('relu', name='relu_6')(norm_6)
    drop_6 = Dropout(0.5, name='drop_6')(relu_6)

    output = Dense(2, activation='softmax', name='fc_7')(drop_6)

    model = Model(inputs=input_, outputs=output)
    model.summary()
#     plot_model(model, to_file='wavelet_cnn_0.5.png')

    return model

In [22]:
model = get_wavelet_cnn_model()

(?, 25152, 80)
(?, 12576, 40, 4)
(?, 2096, 7, 64) (?, 2096, 7, 64)
(?, 350, 2, 128) (?, 350, 2, 128)
(?, 59, 1, 256) (?, 59, 1, 256)


W0220 20:52:48.279892 140650830956352 deprecation.py:506] From /home/rohit/anaconda3/envs/EEG/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 75673, 1)     0                                            
__________________________________________________________________________________________________
sinc_conv1d_7 (SincConv1D)      (None, 75423, 80)    160         the_input[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, 25141, 80)    0           sinc_conv1d_7[0][0]              
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 25141, 80)    320         max_pooling1d_7[0][0]            
__________________________________________________________________________________________________
layer_norm

In [23]:
epochs = 200
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [24]:
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

W0220 20:53:23.745131 140650830956352 module_wrapper.py:137] From /home/rohit/anaconda3/envs/EEG/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [25]:
class chunking(Sequence):
    def __init__(self, data_train, train_labels, batch_size):
        self.data_train = data_train
        self.train_labels = train_labels
        self.batch_size = batch_size
        self.n = 0
        self.max = self.__len__()
        self.train_batch = []
    def __len__(self):
        return np.ceil(int(len(self.data_train) / int(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.data_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.train_labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)
    def __next__(self):
        if self.n >= self.max:
            self.n = 0
        result = self.__getitem__(self.n)
        self.n += 1
        return result

## Section to import labels and trimmed audio of ASVSpoof 2015

In [38]:
train_labels = np.load("/media/rohit/New Volume/codes/SA/backup_hinton/rohit/spoof/spoof/npy_data_asvspoof/train_label.npy")
trimmed_audio = np.load("/media/rohit/New Volume/codes/SA/backup_hinton/rohit/spoof/spoof/npy_data_asvspoof/trimmed_audio.npy", mmap_mode='r')

In [39]:
train_labels=list(train_labels)

train_labels1 = list()
for i in train_labels:
    if i == b'human':
        train_labels1.append(1)
    else:
        train_labels1.append(0)
train_labels = to_categorical(train_labels1[:16000])

## Start training

In [26]:
X_train = np.load("/media/rohit/New Volume/codes/SA/temp_npy/trimmed_audio_train.npy",mmap_mode='r')

In [27]:
y_train = np.load("/media/rohit/New Volume/codes/SA/temp_npy/labels_train.npy",mmap_mode='r')

In [28]:
X_val = np.load("/media/rohit/New Volume/codes/SA/temp_npy/trimmed_audio_val.npy",mmap_mode='r')

In [29]:
y_val = np.load("/media/rohit/New Volume/codes/SA/temp_npy/labels_val.npy",mmap_mode='r')

In [31]:
X_train.shape,X_val.shape

((11200, 75673, 1), (4800, 75673, 1))

## Generator section 

In [32]:
training_batch_generator = chunking(X_train,y_train,2)
validation_batch_generator = chunking(X_val,y_val,2)

In [19]:
train_data_gen = ImageDataGenerator(
	#rescale=1./255,
	shear_range=0.1,
	zoom_range=0.1,
	horizontal_flip=True
)
val_data_gen = ImageDataGenerator(
	#rescale=1./255
)


# def train & test generators
train_generator = train_data_gen.flow(
    X_train,
    y_train,
	batch_size=32)
val_generator = val_data_gen.flow(
	X_val,
	y_val,
    batch_size = 32)

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (11200, 75673, 1))

In [33]:
history = model.fit_generator(
	training_batch_generator,
	steps_per_epoch=11200//2,
	epochs=200,
	validation_data=validation_batch_generator,
    validation_steps=4800//2,
	verbose=True)

W0220 20:53:35.994663 140650830956352 deprecation.py:323] From /home/rohit/anaconda3/envs/EEG/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1428: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
  14/5600 [..............................] - ETA: 4:29:06 - loss: 6.4439 - acc: 0.5000

KeyboardInterrupt: 

In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [16]:
acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(len(acc))

plt.plot(acc, label='training accuracy')
plt.plot(val_acc, label='validation accuracy')
plt.title('Accuracy curve')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

NameError: name 'history' is not defined

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='training loss')
plt.plot(val_loss, label='validation loss')
plt.title('Loss curve')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [10]:
def getModel(input_shape,out_dim):
    #
    inputs = Input(input_shape)
#     print(inputs)
    x = sincnet.SincConv1D(80, 251, 16000)(inputs)


    x = MaxPooling1D(pool_size=3)(x)
    x = BatchNormalization(momentum=0.05)(x)
    x = sincnet.LayerNorm()(x)
    x = LeakyReLU(alpha=0.2)(x)

    #DNN final
    prediction = layers.Dense(out_dim, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=prediction)
    model.summary()
    return model

In [16]:
# input_shape = (n,1)
model = getModel((trimmed_audio[1].shape[0],1),2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 75673, 1)          0         
_________________________________________________________________
sinc_conv1d_5 (SincConv1D)   (None, 75423, 80)         160       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25141, 80)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25141, 80)         320       
_________________________________________________________________
layer_norm_2 (LayerNorm)     (None, 25141, 80)         160       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 25141, 80)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 25141, 2)          162       
Total para

In [24]:
trimmed_audio[1].shape

(75673,)

In [47]:
validation_data = [1,0,0,0,0,0]
do_validation = validation_data!=None

In [48]:
print(do_validation)

True
